## 実行時間が長いMNISTデータセットの分類

- 7_MNIST/Clf_3_MNIST_classify.ipynb を変更（ヒートマップによる混同行列の可視化を削除）
- 各セルの実行時間を表示

以下はIntel修正版との性能比較のための時間計測

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, Y_str = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

# 正解ラベルは文字列で渡されるため数値に変換
Y = Y_str.astype('int')

# 教師データとテストデータに分割
# - データ数が多くCVやグリッドサーチでは時間がかかるので、ここではホールドアウト法を用いる
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

### 分類器をテストする関数 test_model() の定義

1. 教師データを学習
2. 予測値を計算
3. 精度の表示

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

def test_model(model):
    # 教師データを学習
    model.fit(X_train, Y_train)

    # テストデータについてモデルからの予測値を計算
    Y_predict = model.predict(X_test)

    # 精度の表示
    print(classification_report(Y_test, Y_predict, digits=3))

### 高速な分類器

- ナイーブベイズ
- ランダムフォレスト

In [ ]:
%%time
from sklearn.naive_bayes import BernoulliNB

# ナイーブベイズ
model = BernoulliNB()
# 分類器の精度検証
test_model(model)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレスト
model = RandomForestClassifier(n_jobs=-1)
# 分類器の精度検証
test_model(model)

### 実行時間が長い分類器

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

# ロジスティック回帰
model = LogisticRegression(max_iter=2500)
# 分類器の精度検証
test_model(model)

In [ ]:
%%time
from sklearn.svm import SVC

# SVM
model = SVC()
# 分類器の精度検証
test_model(model)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier

# kNN分類器を作成
model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
# 分類器の精度検証
test_model(model)

### パラメータのチューニング

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

# kNN分類器を作成
model = KNeighborsClassifier(n_jobs=-1)

# 探索パラメータ、グリッドの設定
param_range = list(range(1, 10, 1))
grid = [{'n_neighbors': param_range}]

# グリッドサーチの作成、実行
gs = GridSearchCV(model, param_grid=grid, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X, Y)

# 結果の表示
print('best accuracy={:.3f}'.format(gs.best_score_))
print('parameter:', gs.best_params_)